# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# checking the number of rows in the given csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Methodology
I am creating a cluster, followed by creating a keyspace (schema), review the business requirements, design the queries, design the primary key for each of the queries, insert the values into the queries, execute the query to verify the data 

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Apache Cassandra Keyspace

In [24]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandradm 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting the Keyspace

In [25]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandradm')
except Exception as e:
    print(e)

Note: Now we need to create tables to run the following queries. With Apache Cassandra, we model the database tables on the queries you want to run.

## Business Requirements
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1 - SONG_LENGTH_TITLE
### DESCRIPTION
Creating a query that retrieves artist, song title, song length, sessionId and itemInSession 
### PRIMARY KEY DESIGN
Here the primary key is a composite partition key that uniquely identifies each column by combining sessionId and itemInSession
In this composite primary key, the field sessionId is used to partition the data, can be distributed across the nodes and itemInSession is utilized to sort from the results as a clustered key

In [26]:
query1 = "CREATE TABLE IF NOT EXISTS song_length_title"
query1 = query1 + "(artist text, song text, length float, sessionId text, itemInSession text, \
                    PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [27]:
# process the files to insert into the query that will be used for apache cassandra tables
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length_title (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9],float(line[5]),line[8],line[3]))

In [28]:
# execute the query to vertify the data have been inserted into the table

query1a = "select artist, song, length, sessionId, itemInSession from song_length_title where sessionId = '338' and itemInSession = '4'"

try:
    rows = session.execute(query1a)
except Exception as e:
    print(e)

# Using Python's prettytable to format the output 
pt = PrettyTable()
pt.field_names = ["artist","song","length","sessionId","itemInSession"]
for i in rows:
    pt.add_row(i)

print(pt)

+-----------+---------------------------------+--------------------+-----------+---------------+
|   artist  |               song              |       length       | sessionId | itemInSession |
+-----------+---------------------------------+--------------------+-----------+---------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |    338    |       4       |
+-----------+---------------------------------+--------------------+-----------+---------------+


## Query 2 - SONG_USER_INFO
### Description
The following query will retrive the artist name, song title, user id, their names, along with sessionId, itemInSession
### Primary Key Design
Since the requirements clearly stated that the query song should be sorted by itemInSession, I designed the primary key to be a composite/compound key, where,
1. the first portion of a composite key will be a combination of userId and SessionId, used to parition the data, that is to be distributed across the nodes
2. the second potion of the composite key will be itemInSession, a clustered key used to sort the results by

In [32]:
## name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS song_user_info"
query2 = query2 + "(artist text, song text, firstName text, lastName text, userId int,\
                    sessionId text, itemInSession text, \
                    PRIMARY KEY ((userid,sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_info (artist, song, firstName, lastName, userId, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4],int(line[10]),line[8],line[3]))

In [31]:
query2a = "select artist, song, firstName, lastName, userId, sessionId, itemInSession \
            from song_user_info where sessionId = '182' and userId = 10"

try:
    rows = session.execute(query2a)
except Exception as e:
    print(e)

pt = PrettyTable()
pt.field_names = ["artist","song","firstName","lastName","userId","sessionId","itemInSession"]


for i in rows:
    pt.add_row(i)

print(pt)

+-------------------+------------------------------------------------------+-----------+----------+--------+-----------+---------------+
|       artist      |                         song                         | firstName | lastName | userId | sessionId | itemInSession |
+-------------------+------------------------------------------------------+-----------+----------+--------+-----------+---------------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |   10   |    182    |       0       |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |   10   |    182    |       1       |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |   10   |    182    |       2       |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |   10   |    182    |       3       |
+-------------------+--------------------

## Query - allhandsagainsthisown
### Description
This query will retrieve user information for a specific song, called 'All Hands Against His Own'
### Primary key Design
I designed the primary key, in such a way:
1. Paritioning of data will happen across the song so the data is distrubted by song across the nodes
2. Clustering (sorting) will happen on the basis of userId

In [33]:
## All user names (first and last) in music app history who listened to the song 'All Hands Against His Own'

query3 = "CREATE TABLE IF NOT EXISTS allhandsagainsthisown"
query3 = query3 + "(firstName text, lastName text, song text, userId int, \
                    PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [34]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO allhandsagainsthisown (firstName, lastName, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [37]:
query3a = "select firstName, lastName from allhandsagainsthisown WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query3a)
except Exception as e:
    print(e)

# Pretty print the output
pt = PrettyTable()
pt.field_names = ["firstName","lastName"]
for i in rows:
    pt.add_row(i)

print(pt)

+------------+----------+
| firstName  | lastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Dropping the tables before closing out the sessions

In [38]:
query = "drop table song_length_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_user_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table allhandsagainsthisown"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()